In [51]:
# Import necessary libraries
import pandas as pd
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer

In [2]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
# Input data
questions = [
    "What are the essential characteristics of a company, according to the Companies Act, No. 7 of 2007?",
    "What documents or information must accompany an application for a banking license made to the Monetary Board?",
    "Who is considered a 'shareholder' under the Companies Act, No. 7 of 2007?",
    "What is the definition of a 'Bill of Exchange' according to the Bills of Exchange Ordinance?",
    "What are the primary functions and powers of the Securities and Exchange Commission of Sri Lanka?",
    "What conditions must be met for a statement included in a prospectus to be considered 'untrue' under the Companies Act?",
    "What specific information must a trader's receipt to a purchaser include, according to the Consumer Affairs Authority Act, No. 9 of 2003?",
    # "What are the consequences if a licensed commercial bank fails to comply with the liquid asset maintenance requirements under the Banking Act, No. 30 of 1988?",
    # "What does 'incidental expenditure' include for acquiring or realizing an asset under the Inland Revenue Act, No. 24 of 2017?",
    # "What are the key elements a liquidator's winding-up plan must contain for a licensed bank subject to resolution, as per the Banking (Special Provisions) Act, No. 17 of 2023?",
    # "How does the Insolvency Ordinance define the effect of a payment, conveyance, contract, dealing, transaction, or execution made by an insolvent before the filing of a petition for sequestration?",
    # "What is the legal effect of securities held in trust by a central depository under the Securities and Exchange Commission of Sri Lanka Act, No. 19 of 2021?",
    # "What constitutes 'fair use' of a copyrighted work under the Intellectual Property Act, No. 36 of 2003?",
    # "What information does a Director-General or authorized officer of the Inland Revenue Department have access to during an inspection for tax purposes?",
    # "What is the liability of a shareholder to a company under the Companies Act, No. 7 of 2007?",
    # "When does the Electronic Transactions Act, No. 19 of 2006, not apply?",
    # "What are the general rules regarding confidentiality in international arbitration proceedings under the International Arbitration Act 2020?",
    # "What is the rule concerning a company's financial assistance in connection with the purchase of its own shares under the Companies Act, No. 7 of 2007?",
    # "What is the duty of a trustee regarding accounts and information about trust property under the Trusts Ordinance?",
    # "What are the consequences if a person fails to comply with a court order obtained by the Central Bank during a foreign exchange investigation, as per the Foreign Exchange Act, No. 12 of 2017?"
]

original_answers = [
    "The essential characteristics of a company include its legal status and capacity and the different types of companies that can be formed.",
    "An application for a banking license must be in writing and accompanied by: - A copy of the Articles of Association or other foundational documents, along with any proposed amendments. - A statement containing the names, addresses, occupations, and qualifications of the Directors and proposed Directors, and of the Chief Executive Officer. - A copy of the audited balance sheet and profit and loss account of the company. The Monetary Board may also require further information or documents to determine suitability.",
    "In the Companies Act, the term 'shareholder' means a person whose name is entered in the share register as the holder for the time being of one or more shares in the company.",
    "A 'Bill of Exchange' is an unconditional order in writing, addressed by one person to another, signed by the person giving it, requiring the person to whom it is addressed to pay on demand, or at a fixed or determinable future time, a sum certain in money to or to the order of a specified person, or to bearer. An instrument that does not comply with these conditions, or orders an act in addition to money payment, is not a bill of exchange.",
    "The Commission's principal functions and powers include: - Granting licenses to bodies corporate to operate as market institutions and ensuring their proper conduct. - Granting licenses to any person to operate as a market intermediary and ensuring their proper conduct. - Issuing general or specific directives to listed public companies or listed foreign entities. - Regulating takeovers or mergers of listed public companies. - Conducting inquiries and investigations into alleged violations. - Taking steps to mitigate systemic risk to the financial system.",
    "A statement included in a prospectus is deemed to be 'untrue' if it is misleading in the form and context in which it is included. A statement is considered included if it is contained in, or in any report or memorandum appearing on the face of, or by reference incorporated in, or issued with, the prospectus.",
    "Every trader who sells goods must, on demand, issue a receipt to the purchaser setting out: - The date of the sale. - The quantity of goods sold. - The price paid for such quantity. - The nature of the transaction (wholesale or retail). - Any other requirements imposed by law relating to receipt issuance.",
    # "If a licensed commercial bank fails to comply with the liquid asset maintenance provisions, it shall, within such time as determined by the Monetary Board, pay to the Central Bank a charge on the amount of the deficiency calculated at a rate determined by the Monetary Board.",
    # "'Incidental expenditure' incurred by a person in acquiring or realizing an asset includes: - Advertising expenditure, transfer taxes, duties and other expenditure of transfer. - Expenditure of establishing, preserving or defending ownership of the asset. - Remuneration for the services of an accountant, agent, auctioneer, broker, consultant, legal advisor, surveyor or valuer relating to the above expenditures.",
    # "A liquidator's winding-up plan must include: - A current pro forma statement of financial position showing assets and liabilities. - Quarterly statements of income and expenditure for the preceding two years and projected for the succeeding two years. - A progress report on the sale and plans for the sale of major assets. - A report on the judicial or extra-judicial pursuit of claims, including actions for annulment or invalidity of fraudulent conveyances. - A report on offences and illegal activities of directors or officers and actions for compensation from them. - A report on the continuation or termination of ongoing contracts. - A report on the liabilities of the licensed bank and a schedule of expected payments to creditors for the succeeding three months.",
    # "All payments, conveyances, contracts, dealings, transactions, and executions and attachments against lands or goods made or levied bona fide by or with any insolvent before the filing of a petition for sequestration are deemed valid, notwithstanding any prior act of insolvency, provided the other party had no notice of any prior act of insolvency committed by the insolvent at the time of such event. However, this validity does not extend to fraudulent preferences.",
    # "Where the central depository holds securities in trust for its holders of securities, the person for whose benefit those securities are held in trust shall be deemed to be the holder of such securities. This person will also enjoy all rights and privileges and be subject to all duties and obligations related to those securities, as if they were the direct holder.",
    # "'Fair use' includes uses where the purpose and character are for non-profit educational purposes, the nature of the copyrighted work and the amount/substantiality of the portion used are considered, and there is no adverse effect on the potential market or value of the work. Specific instances include: - Reproduction in newspapers or periodicals, broadcasting, or communication to the public of articles on current economic, political, or religious topics (unless expressly reserved). - Reproduction and broadcasting for reporting current events, to the extent justified by the purpose. - Reproduction of political speeches, lectures, or legal proceedings for current information.",
    # "An authorized officer has broad access to information, including the power to: - Make copies of records. - Seize records or other items relevant to an authorized purpose. - Seal records or other items. - Question any person in the premises regarding matters under the Act. - Open and examine any receptacle where documents may be found, and make an inventory. - Operate any computer and take a record of data. This access extends to banks and financial institutions, overriding confidentiality laws.",
    # "The liability of a shareholder to the company is limited to any liability expressly provided for in the articles of the company or under this Act. This does not affect a shareholder's liability to a company under a contract, or for tort, breach of fiduciary duty, or other actionable wrongs committed by the shareholder.",
    # "The Act's provisions do not apply to: - A Bill of Exchange. - A Power-of-Attorney. - A Trust, excluding constructive, implied, and resulting trusts. - A contract for sale or conveyance of immovable property or any interest in such property. - Any other document, act, or transaction specified by the Minister via regulations.",
    # "Unless otherwise agreed by the parties, all documents and matters relating to the arbitration shall be confidential, and no party may publish, disclose, or communicate any information relating to the arbitration proceedings or any awards. However, such information may be disclosed if required by a legal duty, for protecting a legal right or interest, for obtaining legal advice, for enforcing or challenging an award, by court order, or to an adviser of any party.",
    # "A company shall not give financial assistance directly or indirectly for the purpose of, or in connection with, the purchase of shares in the company. This restriction also applies to private companies. However, certain transactions are allowed if the board is satisfied and resolves that giving the assistance is in the company's interest, the terms are fair and reasonable, and the company satisfies the solvency test.",
    # "A trustee is bound to keep clear and accurate accounts of the trust property. Additionally, at all reasonable times, upon the request of the beneficiary, the trustee must furnish full and accurate information as to the amount and state of the trust property.",
    # "If a court makes an order directing an authorized dealer, restricted dealer, or other person to appear before an authorized person or to produce books and documents, and such dealer or person fails to comply with the order forthwith, they commit an offence. Upon conviction, they shall be liable to a fine not exceeding five hundred thousand rupees."
]

mas_answers = [
    "According to the Companies Act, No. 7 of 2007, the essential characteristics of a company include:\n\n*   **Legal Status as a Body Corporate:** A company registered under this Act is a body corporate.\n*   **Legal Capacity:** A company has the capacity to carry on or undertake any business or activity, perform any act, or enter into any transaction, both within and outside Sri Lanka, subject to the provisions of Section 13 of the Act.\n*   **Rights, Powers, and Privileges:** A company possesses all the rights, powers, and privileges necessary for the purposes of carrying on its business or activity and entering into transactions, subject to the provisions of any written law of Sri Lanka or any other country.\n\nReferred Acts and Laws:\n- Companies Act No. 7 of 2007, Section 2(2): \"A company shall have, both within and outside Sri Lanka— (a) subject to the provisions of section 13 of the Act, the capacity to carry on or undertake any business or activity, do any act or enter into any transaction ; and (b) subject to the provisions of any written law of Sri Lanka or of any other country, all the rights, powers and privileges necessary for the purposes of paragraph (a).\"",
    "An application for a banking license made to the Monetary Board must be accompanied by specific documents and information, depending on whether the company is newly formed or already established and intending to commence banking business.\n\nFor a company to be formed for the purpose of carrying on banking business:\n*   A copy of the draft Articles of Association or the draft Constitution, or any other document related to the company's formation.\n*   A statement providing the names, addresses, occupations, and qualifications of the proposed Directors. If a Chief Executive Officer has been identified, their details must also be included.\n\nFor a company formed before the date of the application and intending to commence banking business:\n*   A copy of the Articles of Association or the Constitution of such company.\n\nAdditionally, upon receiving an application, the Director of Bank Supervision may request any other documents, information, or particulars deemed necessary to determine whether to issue a license.",
    "Under the Companies Act, No. 7 of 2007, a \"shareholder\" is defined as:\n\n*   A person whose name is currently entered in the share register as the holder of one or more shares in the company.\n*   A person who is named as a shareholder in an application for the company's incorporation at the time of its registration, until their name is entered in the share register.\n*   A person who is entitled to have their name entered in the share register as a shareholder in an amalgamated company under a registered amalgamation proposal, until their name is entered.\n\nReferred Acts and Laws:\n*   Companies Act No. 7 of 2007, Section 86(1)",
    "According to the Bills of Exchange Ordinance:\n\nA 'Bill of Exchange' is defined as an unconditional order in writing, addressed by one person to another, and signed by the person giving it. This order requires the person to whom it is addressed to pay, either on demand or at a fixed or determinable future time, a sum certain in money to or to the order of a specified person, or to the bearer.\n\nAn instrument is not considered a bill of exchange if it does not comply with these conditions or if it orders any act to be done in addition to the payment of money.\n\nAn order to pay out of a particular fund is not considered unconditional. However, an unqualified order to pay, when coupled with either an indication of the account to be debited with the amount or a statement of the transaction that gives rise to the bill, is deemed unconditional.\n\nA bill is not rendered invalid merely because it is not dated, does not specify the value given, or the place where it is drawn or payable.\n\nReferred Acts and Laws:\n*   Bills of Exchange Ordinance No. 82, Sections 3(1), 3(2), and 3(3)",
    "The primary functions and powers of the Securities and Exchange Commission of Sri Lanka (SEC), as outlined in the Securities and Exchange Commission of Sri Lanka Act No. 19 of 2021, are as follows:\n\n**Primary Functions (Objects and Purpose):**\n*   To establish the Securities and Exchange Commission of Sri Lanka.\n*   To create, maintain, and regulate a fair, orderly, efficient, and transparent securities market.\n*   To protect the interests of both local and foreign investors.\n*   To ensure the maintenance of high professional standards in the provision of services related to securities markets.\n\n**Powers and Duties:**\n*   To advise the Government on the development of the securities market and assist in implementing policies.\n*   To encourage and promote the development of securities markets in Sri Lanka.\n*   To issue directives to market participants.\n*   To conduct supervision or inspections of market institutions and intermediaries.\n*   To exempt public offers from the Act’s provisions.\n*   To appoint experts.\n*   To perform all acts necessary or incidental to the Commission’s objectives.\n\nReferred Acts and Laws:\n*   Securities and Exchange Commission of Sri Lanka Act No. 19 of 2021, Sections 3, 16, and 17",
    "According to the Consumer Affairs Authority Act, No. 9 of 2003, every trader who sells goods must, upon demand, issue a receipt to the purchaser that includes the following specific information:\n\n*   The date of the sale.\n*   The quantity of goods sold.\n*   The price paid for that quantity.\n*   The nature of the transaction (wholesale or retail).\n*   Any other requirements imposed under any other law relating to receipts.\n\nReferred Acts and Laws:\n*   Consumer Affairs Authority Act No. 9 of 2003, Section 28",
    "If a licensed commercial bank fails to comply with the liquid asset maintenance requirements under the Banking Act, No. 30 of 1988, which are determined by the Central Bank, the Monetary Board may give notice that it would cancel the license issued to such bank. This action can be taken if the bank has contravened any provisions of the Act or any direction, order, or other requirement imposed under it.\n\nReferred Acts and Laws:\n- Banking Act No. 30 of 1988, Section 21"
]


chatgpt_answers = [ 
    "According to Section 4 of the Companies Act, No. 7 of 2007, a company must: (a) be incorporated under the Act, (b) have a name, (c) have one or more shareholders, (d) have one or more directors, and (e) have a registered office in Sri Lanka.",
    "According to Section 5 of the Banking Act, No. 30 of 1988, an application for a banking license must include: (a) the company’s incorporation documents, (b) business plan, (c) particulars of directors and shareholders, (d) audited financial statements, and (e) any additional documents required by the Monetary Board.",
    "A 'shareholder' is defined in Section 86 of the Companies Act, No. 7 of 2007, as a person whose name is entered in the share register of the company as the holder of one or more shares.",
    "As per Section 3 of the Bills of Exchange Ordinance, a 'Bill of Exchange' is an unconditional order in writing addressed by one person to another, signed by the person giving it, requiring the addressee to pay a sum certain in money to, or to the order of, a specified person or to bearer.",
    "Under Section 12 of the Securities and Exchange Commission of Sri Lanka Act, No. 19 of 2021, the Commission’s primary functions include: regulating the securities market, granting licenses to market institutions, protecting investors, and promoting market integrity and transparency.",
    "According to Section 40(1)(a) of the Companies Act, No. 7 of 2007, a statement in a prospectus is considered 'untrue' if it is misleading in the form and context in which it is included, or if there is a material omission that renders the statement misleading.",
    "As per Section 12(2) of the Consumer Affairs Authority Act, No. 9 of 2003, a trader’s receipt to a purchaser must include: (a) the date of transaction, (b) the name and address of the trader, (c) description of goods or services sold, and (d) the price paid.",
    # "According to Section 20 of the Banking Act, No. 30 of 1988, if a licensed commercial bank fails to maintain the required liquid assets, the Monetary Board may impose penalties, issue directions, or take measures including suspension or cancellation of the license.",
    # "As per Section 21 of the Inland Revenue Act, No. 24 of 2017, 'incidental expenditure' includes legal, valuation, and advertising costs, fees, commissions, and similar expenses incurred directly in acquiring or realizing an asset.",
    # "Under Section 20 of the Banking (Special Provisions) Act, No. 17 of 2023, a liquidator’s winding-up plan must include: (a) a statement of assets and liabilities, (b) recovery strategy, (c) distribution to creditors, (d) timelines, and (e) legal and regulatory compliance steps.",
    # "According to Section 91 of the Insolvency Ordinance, any transaction made by an insolvent prior to the filing of a petition may be declared void if made with intent to defraud creditors or if it constitutes a fraudulent preference.",
    # "Under Section 24 of the Securities and Exchange Commission Act, No. 19 of 2021, securities held in trust by a central depository are deemed to be held on behalf of the beneficial owners and not as part of the depository’s own assets.",
    # "Section 12 of the Intellectual Property Act, No. 36 of 2003 defines 'fair use' to include use for purposes such as criticism, comment, news reporting, teaching, scholarship, or research, provided it does not conflict with the normal exploitation of the work.",
    # "Under Section 127 of the Inland Revenue Act, No. 24 of 2017, the Director-General or authorized officers may inspect business premises and have access to books, records, and electronic data necessary for verifying tax compliance.",
    # "As per Section 30 of the Companies Act, No. 7 of 2007, a shareholder’s liability to the company is limited to the amount unpaid on their shares.",
    # "According to Section 2(2) of the Electronic Transactions Act, No. 19 of 2006, the Act does not apply to: (a) wills, (b) negotiable instruments, (c) powers of attorney, (d) contracts for sale or transfer of immovable property.",
    # "Section 23 of the International Arbitration Act, No. 5 of 2020, provides that arbitration proceedings are confidential unless otherwise agreed, or disclosure is required by law or for the enforcement of an award.",
    # "According to Section 101 of the Companies Act, No. 7 of 2007, a company shall not provide financial assistance for the purchase of its own shares unless it complies with solvency requirements and passes the necessary board and shareholder resolutions.",
    # "Under Section 24 of the Trusts Ordinance, a trustee must maintain accurate accounts and provide beneficiaries with information concerning the trust property upon request, unless unreasonable.",
    # "As per Section 33 of the Foreign Exchange Act, No. 12 of 2017, failure to comply with a court order obtained during an investigation may result in contempt of court proceedings and penalties under the Act."
]

gemini_answers = [
    "According to Section 4 of the Companies Act, No. 7 of 2007, a company must: (a) be incorporated under the Act, (b) have a name, (c) have one or more shareholders, (d) have one or more directors, and (e) have a registered office in Sri Lanka.",
    "As per Section 5 of the Banking Act, No. 30 of 1988, an application for a banking license must be accompanied by: (a) certified copies of incorporation documents, (b) details of directors and shareholders, (c) business plan and financial projections, (d) feasibility study, and (e) any further information required by the Monetary Board.",
    "Under Section 86 of the Companies Act, No. 7 of 2007, a shareholder is a person whose name is entered in the share register of the company as the holder of one or more shares.",
    "According to Section 3 of the Bills of Exchange Ordinance, a bill of exchange is an unconditional order in writing, addressed by one person to another, signed by the person giving it, requiring the person to whom it is addressed to pay on demand or at a fixed or determinable future time a sum certain in money to order or to bearer.",
    "Under Section 13 of the Securities and Exchange Commission of Sri Lanka Act, No. 19 of 2021, the SEC is empowered to regulate the securities market, grant licenses to market participants, monitor trading activities, enforce compliance with securities law, and protect investor interests.",
    "Under Section 40(1) of the Companies Act, No. 7 of 2007, a statement is deemed 'untrue' if it is misleading in the form and context in which it is included, or if it omits a material fact that is necessary to prevent the statement from being misleading.",
    "As per Section 26(2) of the Consumer Affairs Authority Act, No. 9 of 2003, a receipt issued by a trader to a purchaser must include: (a) the date of sale, (b) the description of goods sold, (c) the quantity sold, (d) the price of each item, and (e) the total price.",
    # "Under Section 20(3) of the Banking Act, No. 30 of 1988, if a licensed commercial bank fails to maintain the required liquid assets, the Monetary Board may impose penalties, restrict operations, or even suspend the license of the bank.",
    # "According to Section 35(3) of the Inland Revenue Act, No. 24 of 2017, incidental expenditure for acquiring or realizing an asset includes: (a) legal fees, (b) stamp duty, (c) brokerage, and (d) other similar expenses directly related to the acquisition or realization.",
    # "Under Section 19 of the Banking (Special Provisions) Act, No. 17 of 2023, a liquidator's winding-up plan must include: (a) estimated realizable value of assets, (b) priority of payments, (c) proposed timeline for winding up, and (d) impact on depositors and creditors.",
    # "Section 94 of the Insolvency Ordinance states that any payment, conveyance, contract, or transaction made by a person unable to pay debts, before a petition for sequestration, with the intention to prefer one creditor over another, is void against the assignee.",
    # "Section 62(2) of the Securities and Exchange Commission of Sri Lanka Act, No. 19 of 2021 provides that securities held in trust by a central depository are immune from claims by creditors of the depository and are held for the benefit of account holders.",
    # "According to Section 11 of the Intellectual Property Act, No. 36 of 2003, fair use includes use for purposes such as criticism, comment, news reporting, teaching, scholarship, or research, provided it does not unreasonably prejudice the rights of the author.",
    # "Section 164(1) of the Inland Revenue Act, No. 24 of 2017 empowers the Director-General or authorized officer to access any books, records, accounts, and electronic data during inspection for determining tax liability.",
    # "Section 69 of the Companies Act, No. 7 of 2007 provides that a shareholder's liability to a company is limited to: (a) any amount unpaid on shares held by them, and (b) any liability expressly provided in the company's constitution or a separate agreement.",
    # "As per Section 2(2) of the Electronic Transactions Act, No. 19 of 2006, the Act does not apply to: (a) the creation or execution of a will, (b) negotiable instruments, (c) power of attorney, and (d) documents required by law to be notarized.",
    # "Section 31 of the International Arbitration Act, No. 5 of 2020, states that the parties must keep the arbitral proceedings confidential unless otherwise agreed or required by law, including documents, evidence, and the award.",
    # "Section 101 of the Companies Act, No. 7 of 2007 states that a company shall not provide financial assistance for the purpose of acquiring its own shares except under limited exceptions such as employee share schemes or financial assistance not materially reducing the company’s net assets.",
    # "Section 13 of the Trusts Ordinance imposes a duty on the trustee to maintain accurate accounts and provide complete and true information concerning the trust property to the beneficiaries upon reasonable request.",
    # "Under Section 33 of the Foreign Exchange Act, No. 12 of 2017, a person who fails to comply with a court order obtained by the Central Bank during a foreign exchange investigation commits an offence and is liable on conviction to a fine or imprisonment or both."
]

grok_answers = [
    "A company is a separate legal entity with perpetual succession, capable of owning property, entering contracts, and suing or being sued. It must be registered, have a memorandum and articles of association, and comply with share capital, directors, and statutory filing requirements.",
    "A banking license application requires a business plan, financial projections, promoter and director details (qualifications, experience), capital structure, financial capability evidence, incorporation details, anti-money laundering policies, and risk management frameworks.",
    "A shareholder is a person whose name is entered in the company’s share register as the holder of one or more shares.",
    "A Bill of Exchange is an unconditional written order by the drawer to the drawee, signed by the drawer, requiring payment of a sum certain in money on demand or at a fixed future time to a specified person or bearer.",
    "The SEC regulates the securities market, licenses participants, protects investors, promotes market development, and enforces compliance. It can issue rules, investigate violations, impose penalties, and suspend or cancel licenses.",
    "A prospectus statement is untrue if it is misleading in context or contains material omissions affecting its import, rendering it false or deceptive.",
    "A trader’s receipt must include the trader’s name and address, date of sale, description of goods or services, quantity, price, and total amount paid.",
    # "Failure to maintain liquid assets may lead to penalties, corrective directives, operational restrictions, license suspension, or regulatory intervention.",
    # "Incidental expenditure includes legal fees, valuation fees, stamp duties, advertising costs, and other expenses directly related to acquiring or disposing of an asset, excluding the asset’s purchase price or realization proceeds.",
    # "A liquidator’s winding-up plan must include asset and liability assessment, liquidation strategy, creditor claim prioritization, asset recovery measures, a timeline, and regulatory compliance to protect depositors and financial stability.",
    # "Payments, conveyances, contracts, or transactions by an insolvent before sequestration may be void or voidable if deemed fraudulent preferences or transfers to defeat creditors, as determined by the court.",
    # "Securities held in trust by a central depository are for the benefit of the beneficial owner, with the depository acting as a nominee, ensuring secure custody, transfer, and settlement, while the owner retains dividend, voting, and other rights.",
    # "Fair use includes private study, research, criticism, review, reporting, or teaching, provided the use is limited, does not harm the original work’s market, and is proportionate to the purpose.",
    # "The Director-General or authorized officer may access books of accounts, records, documents, electronic data, premises, and other relevant information or assets to verify tax compliance or investigate evasion.",
    # "A shareholder’s liability is limited to the amount unpaid on their shares, with no personal liability for company debts beyond their share capital contribution, unless personally guaranteed.",
    # "The Electronic Transactions Act does not apply to wills, codicils, trusts, powers of attorney, immovable property sales, negotiable instruments (except electronic cheques), or other excluded transactions.",
    # "International arbitration proceedings are generally confidential unless parties agree otherwise, covering documents, evidence, and awards, subject to legal requirements, court orders, or public interest exceptions.",
    # "A company cannot provide financial assistance for purchasing its own shares, except for employee share schemes, ordinary business loans, or with special resolution and solvency test approval.",
    # "A trustee must keep accurate accounts of trust property and provide beneficiaries with reasonable information about trust administration and property upon request, subject to trust terms and good faith duties.",
    # "Failure to comply with a court order in a foreign exchange investigation may result in fines, imprisonment up to 7 years, asset seizure, or additional regulatory sanctions."
]

deepseek_answers = [
    "Section 4 of the Companies Act No. 7 of 2007 establishes that a company must possess several fundamental characteristics: it must be a separate legal entity distinct from its members, have limited liability for shareholders, enjoy perpetual succession regardless of ownership changes, maintain transferable shares (unless restricted), and have the capacity to enter contracts and litigate in its own name. These features collectively define the corporate form under Sri Lankan law.",
    "Section 8 of the Banking Act No. 30 of 1988 mandates that banking license applications to the Monetary Board must include: the company's constitutional documents (MOA/AOA), three-year financial projections demonstrating viability, detailed profiles of proposed directors meeting fit-and-proper criteria, comprehensive business plans covering operations and risk management frameworks, and evidence of meeting minimum capital requirements (Rs. 2.5 billion for commercial banks as of 2023).",
    "Section 2 of the Companies Act defines a shareholder as any natural or legal person registered in the company's share register as holding one or more shares. This includes both legal owners and beneficial owners where shares are held through nominees, with the registered holder being legally recognized unless a disclosure of beneficial ownership is made under Section 134 of the Act.",
    "Section 3 of the Bills of Exchange Ordinance (Chapter 82) defines a bill of exchange as an unconditional written order signed by the drawer, directed to the drawee, requiring payment of a specified sum to a named payee or bearer either on demand or at a fixed future date. The instrument must contain all essential elements including certainty of parties, amount, and payment terms to be valid under Sri Lankan law.",
    "Section 10 of the SEC Act No. 19 of 2021 empowers the Securities and Exchange Commission to: regulate stock exchanges and securities markets; license and supervise market intermediaries; ensure proper disclosure and investor protection; investigate market misconduct; make rules for orderly markets; and promote capital market development. These functions are exercised under the overall supervision of the Minister of Finance.",
    "Section 48 of the Companies Act provides that a prospectus statement is 'untrue' if it: contains material misstatements of fact; omits material information making disclosed facts misleading; or presents information in a manner likely to deceive investors regarding share subscription decisions. Liability attaches regardless of whether the misrepresentation was intentional or negligent.",
    "Section 25 of the Consumer Affairs Authority Act No. 9 of 2003 requires trader receipts to contain: seller's name/address/registration number; buyer's name (if applicable); detailed description of goods/services; quantity/measurements; unit price and total amount; applicable taxes; date of supply; warranty particulars (if any); and return policy details. Electronic receipts must contain equivalent information.",
    # "Section 76 of the Banking Act imposes several sanctions for liquid asset maintenance failures: daily penalty interest on deficiencies; restrictions on dividend payments; increased capital adequacy requirements; mandatory corrective action plans; and potential suspension of certain banking activities. Repeated violations may lead to license revocation under Section 77 after proper inquiry by the Monetary Board.",
    # "Section 23 of the Inland Revenue Act No. 24 of 2017 defines incidental expenditure for assets as including: legal/conveyancing fees; valuation/appraisal costs; brokerage/agency commissions; transfer duties/stamp fees; transportation/insurance for physical assets; installation/commissioning expenses; and other directly attributable costs of acquisition or disposal. These are deductible when calculating capital gains.",
    # "Section 12 of the Banking (Special Provisions) Act No. 17 of 2023 mandates that winding-up plans must include: asset/liability valuations; creditor classification and priorities; proposed recovery strategies; employee settlement terms; depositor protection measures; timelines for asset realization; distribution mechanisms; and contingency arrangements. Plans require Central Bank approval before implementation.",
    # "Section 54 of the Insolvency Ordinance provides that transactions within 3 months preceding sequestration may be voided if: made with intent to defraud creditors; constituting undue preference to specific creditors; or executed when insolvent without fair consideration. The burden of proving bona fides rests with the party seeking to uphold such transactions.",
    # "Section 29 of the SEC Act No. 19 of 2021 establishes that securities held in central depositories: are dematerialized and fungible; create no proprietary interest for the depository; are protected from third-party claims; and confer direct rights on beneficial owners as recorded in depository accounts, with ownership transfers effected through book-entry changes.",
    # "Section 12 of the Intellectual Property Act No. 36 of 2003 permits fair use for: research/private study; criticism/review; news reporting; judicial proceedings; educational instruction; and incidental inclusion. The provision balances copyright protection with public interest considerations, assessing factors like purpose, nature, amount used, and market effect.",
    # "Section 130 of the Inland Revenue Act authorizes officers to access: accounting records; bank statements; contracts/agreements; asset registers; electronic data systems; and third-party information relevant to tax liability. Officers may make copies, extracts, and require explanations during inspections conducted with reasonable notice.",
    # "Section 36 of the Companies Act limits shareholder liability to: unpaid amounts on subscribed shares; personal guarantees given to creditors; and situations where courts pierce the corporate veil due to fraud, improper conduct, or failure to maintain legal formalities between company and shareholders.",
    # "Section 4 of the Electronic Transactions Act excludes: wills/codicils; trusts; powers of attorney; negotiable instruments; documents requiring notarization; and transactions involving immovable property. These exceptions preserve traditional formalities for sensitive legal instruments under general law.",
    # "Section 22 of the International Arbitration Act 2020 establishes confidentiality for: all arbitration proceedings; evidence/documents submitted; awards issued; and settlement terms. Disclosure requires party consent or court order, with exceptions for legal obligations, professional duties, or necessary enforcement proceedings.",
    # "Section 58 of the Companies Act prohibits financial assistance for share acquisitions except when: approved by special resolution; fair to creditors; properly disclosed; and either part of employee share schemes or ordinary business activities. Violations render the assistance void and may attract director liability.",
    # "Section 81 of the Trusts Ordinance requires trustees to: maintain accurate accounts; provide beneficiaries with full information upon request; render annual accounts (unless waived); keep trust property distinct; and allow reasonable beneficiary inspection of trust documents and assets.",
    # "Section 35 of the Foreign Exchange Act imposes for non-compliance with investigation orders: fines up to Rs. 1 million; imprisonment up to 1 year; or both. Continued non-compliance after conviction attracts additional daily fines. Courts may enforce compliance through contempt proceedings."
]

In [20]:
# Encode original answers once
original_vecs = [model.encode([ans])[0] for ans in original_answers]

# Helper function using scipy cosine distance
def cosine_sim(vec1, vec2):
    return 1 - distance.cosine(vec1, vec2)

answer_dict = {
    'MAS': mas_answers,
    'Grok': grok_answers,
    'DeepSeek': deepseek_answers,
    'Gemini': gemini_answers,
    # 'RAG': rag_answers
}

In [21]:
results = []
for i, orig_vec in enumerate(original_vecs):
    row = {'Question': questions[i]}
    for method, answers in answer_dict.items():
        method_vec = model.encode([answers[i]])[0]
        row[method] = cosine_sim(orig_vec, method_vec)
    results.append(row)

df = pd.DataFrame(results)
df.set_index('Question', inplace=True)
avg_scores = df.mean(axis=0).to_frame(name='Average Similarity Score')

In [23]:
# Display the results in a DataFrame
df

,MAS,Grok,DeepSeek,Gemini
Question,,,,
"What are the essential characteristics of a company, according to the Companies Act, No. 7 of 2007?",0.531979,0.632964,0.681697,0.430841
What documents or information must accompany an application for a banking license made to the Monetary Board?,0.898004,0.837916,0.682798,0.861024
"Who is considered a 'shareholder' under the Companies Act, No. 7 of 2007?",0.899792,0.929126,0.795137,0.896859
What is the definition of a 'Bill of Exchange' according to the Bills of Exchange Ordinance?,0.817742,0.921147,0.842650,0.937811
What are the primary functions and powers of the Securities and Exchange Commission of Sri Lanka?,0.532625,0.556164,0.554465,0.537072
What conditions must be met for a statement included in a prospectus to be considered 'untrue' under the Companies Act?,0.188699,0.829762,0.778762,0.721742
"What specific information must a trader's receipt to a purchaser include, according to the Consumer Affairs Authority Act, No. 9 of 2003?",0.243504,0.755861,0.697718,0.804980


In [13]:
avg_scores

,Average Similarity Score
Grok,0.766360
DeepSeek,0.692223
Gemini,0.726759
